In [40]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [41]:
# entrar no index.html
import os

caminho = os.getcwd()
arquivo = caminho + r"\index.html"
navegador.get(arquivo)

In [42]:
# escolher um estado
from selenium.webdriver import ActionChains

botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
ActionChains(navegador).move_to_element(botao).perform()

navegador.find_element(By.XPATH, '/html/body/div/div/div/a[2]').click()

In [43]:
# mudar para a nova aba
aba_original = navegador.window_handles[0]
nova_aba = navegador.window_handles[1]

navegador.switch_to.window(nova_aba)
# preencher o formulário com os dados da busca
navegador.find_element(By.ID, 'nome').send_keys('Gabriel Cabral')
navegador.find_element(By.ID, 'advogado').send_keys('Cabral Gabriel')
navegador.find_element(By.ID, 'numero').send_keys('4820')

# clicar em pesquisar
navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

In [44]:
# confirmar a pesquisa
alerta = navegador.switch_to.alert
alerta.accept()

In [45]:
# esperar o resultado da pesquisa e agir de acordo com o resultado
import time

while True:
    try:
        alerta = navegador.switch_to.alert
        break
    except:
        time.sleep(1)
        
texto_alerta = alerta.text
if 'Processo encontrado com sucesso' in texto_alerta:
    alerta.accept()
    print('Processo foi encontrado!')
else:
    alerta.accept()
    print('Processo não encontrado!')

Processo não encontrado!
